TF-IDF
---
* DTM내에 있는 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF 가중치에 대해 알아본다.
* TF-IDF를 사용하면, 기존 DTM을 사용하는 것보다 더 많은 정보를 고려하여 문서들을 비교할 수 있다. 항상 성능이 뛰어나다는 소리는 아님.

1. TF-IDF(단어 빈도-역 문서 빈도)
* DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방법. DTM을 우선 만들고, TF-IDF 가중치를 부여
* TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업등에 쓰일 수 있다.
* TF와 IDF를 곱한 값을 의미하는데 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현 할 때 TF, DF, IDF는 각각 다음과 같이 정의 한다.
    - (1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
    - (2) df(f) : 특정 단어 t가 등장한 문서의 수 
    - (3) idf(d,t) : df(t)에 반비례하는 수
                idf(d, t) = log({n}/{1+df(t)})
                IDF는 DF의역수를 취하고 싶은 것이 맞다. log와 분모에 1을 더해주는 것은 log를 사용하지 않으면 n이 커질수록 기하급수적으로 커지기 때문이다.
                log를 씌우지 않으면 희귀 단어들에 엄청난 가중치가 부여될수도 있어서 씌워서 격차를 줄이는 효과를 얻는다.
                1을 더하는 이유는 특정 단어가 전체 문서에서 등장하지 않으면 분모가 0이 되는 경우를 방지하기 위해서 이다.


* 파이썬으로 TF-IDF 직접 구현하기

In [2]:
import pandas as pd
from math import log #IDF계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [3]:
#tf, idf, tf-idf 구하는 함수 
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [5]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_  #DTM을 구했다

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [6]:
#IDF를 구해보자
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [7]:
#TF-IDF를 구해보자
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [9]:
#사이키럿을 통한 DTM과 TF-IDF 실습

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
#DTM 완성

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [10]:
#TfidVectorizer를 통해 자동으로 TF-IDF 계산
#값이 조금 달라도 (L2 정규화 같은 것 때문에 ) TF-IDF가진 의도를 그대로 갖고있어서 사이킷런으로 사용해도 된다.
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
